In [1]:
#Import packages
import io
import re
import pickle
import numpy as np
import networkx as nx
import operator
from collections import Counter

In [2]:
# Get all the episodes' names in season.txt
f1 = io.open('season.txt', 'r', encoding='utf-8')
text = f1.read()
episodesname = re.findall(r'\[\[(.*?)[\|\]\]]', text)
f1.close()

## Create function that calculates the sentiment of a text using the LabMT file

In [3]:
# Load the LabMT file which is the file containing the sentiment words and their score
f1 = io.open('LabMT.txt', 'r' ,encoding='utf-8')
wlist=f1.read().split()
wordlist=wlist[15:len(wlist)]

In [4]:
def sentiment(textfile):
    if not isinstance(textfile,list):
        #Load the textfile that should be analyzed
        text1 = io.open(textfile, 'r',encoding='utf-8')
        textlist=text1.read().split()
        text1.close()
    else:
        textlist=textfile
    #Remove all numbers.
    nonumbers=re.compile("[^\d]+").findall(' '.join(textlist))
    #Remove everything that is not words and return their lowercase.
    onlywords=re.compile("[\w|']+").findall(' '.join(nonumbers))
    finallist=[w.lower() for w in onlywords]

    #Make a list of all the words in the textfile and their count
    counts = Counter(finallist)
    most_common_list=counts.most_common()

    #Set the score to 0 and the word count to 0.
    score=0
    wordcount=0
    for i in range(len(most_common_list)):
        #Get a word in the textfile that is being scored.
        curword=most_common_list[i]
        #If the list in the text is part of the sentiment wordlist do the following.
        if curword[0] in wordlist:
            #Find the index of the word in the wordlist
            #and then add the score times the number of times the words appear to score
            wordindex=wordlist.index(curword[0])
            #The score is always 2 places further down in the list compared to the word.
            if wordlist[wordindex+2]!='--':
                score+=float(wordlist[wordindex+2])*curword[1]
                wordcount+=curword[1]
    if score==0:
        if not isinstance(textfile,list):
            print "None of the words in %s was in the LabMT wordlist." %textfile
        else:
            print "None of the words in the text file was in the LabMT wordlist."
    else:
        return score/wordcount

## Character sentiment score
* Count the frequency of the characters in each episodes

* If the character appears 3 or more times, calculate the sentiment score and give the score to the character

* Store the score of each character and average them


In [5]:
#Get the character sentiment scores.
charnamesopen= open('char_names.txt', 'r')
charlist=charnamesopen.read().splitlines()
charnamesopen.close()
urlnamesopen= open('url_names.txt', 'r')
urllist=urlnamesopen.read().splitlines()
urlnamesopen.close()
season = 0
sentdict={}

# Give the character a sentiment score if they appear 3 or more times.
for i in range(len(episodesname)):
    if episodesname[i][-5:] == "recap":
        continue
    if episodesname[i][:6] == "Season":
        season += 1
        print "-----season %d-----" % season
        continue
    if season != 0:
        #Use the clean text for the sentiment score put the html text for finding characters.
        text = "Episodes/%s.txt" % episodesname[i]
        text1 = open("Episodes_api/%s.txt" % episodesname[i]).read()
        textsent=sentiment(text)
        for p in range(len(charlist)):
            url=urllist[p]
            #Find the number of times a characters name appear in an episode using a regular expression.
            regexname=r"\/" + re.escape(url) + r"\b"
            match=re.findall(regexname, text1)
            #If the number of matches is 3 or more, add the score to the character
            if len(match)>=3:
                #If the character is not already in the dictionary, then add it.
                if charlist[p] not in sentdict.keys():
                    sentdict[charlist[p]]=[]
                sentdict[charlist[p]].append(textsent)
print "done"

-----season 1-----
-----season 2-----
-----season 3-----
-----season 4-----
-----season 5-----
-----season 6-----
done


In [15]:
#Create a new dictionary where the characters sentiment scores are stored as the average of all their sentiment scores,
#that were calculated above
charkeys=sentdict.keys()
chardictmean={}
for name in charkeys:
    chardictmean[name]=float("{0:.5f}".format(np.mean(sentdict[name])))

#Sort the dictionary so the highest scores appear first.
char_sent_sorted=sorted(chardictmean.items(), key=operator.itemgetter(1),reverse=True)
char_sent_sorted

[('Jhiqui', 5.38299),
 ('Wylla', 5.38299),
 ('Lady', 5.38299),
 ('Catspaw assassin', 5.38299),
 ('Lothar Frey', 5.37487),
 ('Prendahl na Ghezn', 5.37063),
 ('Mero', 5.37063),
 ('Pentoshi servant', 5.36387),
 ('Waymar Royce', 5.36387),
 ('Wildling girl', 5.36387),
 ('Gared', 5.36387),
 ('Will', 5.36387),
 ('Tommy', 5.36387),
 ('Ilyn Payne', 5.36148),
 ('Cressen', 5.36022),
 ('Lothor Brune', 5.36022),
 ("Captain's daughter", 5.35936),
 ('Ros', 5.35732),
 ('Rorge', 5.35542),
 ('Biter', 5.35542),
 ('Doreah', 5.35301),
 ('Axell Florent', 5.35231),
 ('Tansy', 5.35231),
 ('Dying man', 5.35149),
 ('Rhaego', 5.34961),
 ('Tom', 5.34961),
 ('Morag', 5.34812),
 ('Jon Arryn', 5.34515),
 ('Aegon V Targaryen', 5.34404),
 ('Viserys Targaryen', 5.34395),
 ('Tobho Mott', 5.34333),
 ('Rakharo', 5.3432),
 ('Royal Steward', 5.3432),
 ('Vardis Egen', 5.34156),
 ('Stiv', 5.34156),
 ('Mord', 5.34156),
 ('Lyonel Baratheon', 5.34156),
 ('Steffon Baratheon', 5.34156),
 ('Orys Baratheon', 5.34156),
 ('Wallen', 5.

In [16]:
print len(char_sent_sorted)

302


* After we've done this, consider their sentiment score with attributes like their houses, dead/alive. See if a dead person has lower sentiment score, and/or compare the sentiment average score of different houses

In [20]:
#Load directed graph, undirected graph and the list of philosophers.
dg = nx.read_gpickle("Digraph_GOT_with_attributes.gpickle")

#Generate the largest connected component.
subdg = list(nx.weakly_connected_component_subgraphs(dg))
dg=subdg[0]


In [21]:
dead=[]
alive=[]
#Use the attribute "Status" of the graphs to determine if a character is dead or alive.
for name in sentdict.keys():
    if nx.get_node_attributes(dg,"Status")[name]=="Dead":
        dead.append(np.mean(sentdict[name]))
    if nx.get_node_attributes(dg,"Status")[name]=="Alive":
        alive.append(np.mean(sentdict[name]))

print "Mean sentiment of dead characters: %f" % np.mean(dead)
print "Mean sentiment of Alive characters: %f" % np.mean(alive)

Mean sentiment of dead characters: 5.315106
Mean sentiment of Alive characters: 5.313668


In [22]:
#Find the sentiment score of the houses/allegiance by using the graph attributes "House"
allhouses=nx.get_node_attributes(dg,"House").values()
unique_houses = list(set(x for l in allhouses for x in l))
housesentdict={}

for name in sentdict.keys():
    for house in nx.get_node_attributes(dg,"House")[name]:
        if house not in housesentdict.keys():
            housesentdict[house]=[]
        housesentdict[house].append(np.mean(sentdict[name]))

In [24]:
#Create a new dictionary where the scores are averaged.
housekeys=housesentdict.keys()
housedictmean={}
for name in housekeys:
    housedictmean[name]=float("{0:.5f}".format(np.mean(housesentdict[name])))
    
house_sent_sorted=sorted(housedictmean.items(), key=operator.itemgetter(1),reverse=True)
house_sent_sorted

[(u'Sansa Stark', 5.38299),
 (u'Illyrio Mopatis', 5.36387),
 (u'Dragonstone', 5.36022),
 (u'House Brune', 5.36022),
 (u'Second Sons (mercenary company)', 5.358),
 (u'Xaro Xhoan Daxos', 5.35301),
 (u'Craster', 5.34812),
 (u'House Egen', 5.34156),
 (u'Khal', 5.34148),
 (u'House Payne', 5.34136),
 (u'House Dayne', 5.34101),
 (u'House Slynt', 5.33783),
 (u'Moro', 5.33771),
 (u'Drogo', 5.33737),
 (u'Varys', 5.3367),
 (u'House Cerwyn', 5.33623),
 (u'House Stokeworth', 5.33608),
 (u'House Hollard', 5.33564),
 (u'Three-eyed raven', 5.33527),
 (u'Winterfell', 5.33388),
 (u'Maegyr family', 5.33305),
 (u'Warlocks of Qarth', 5.33217),
 (u'House Cassel', 5.32953),
 (u'First Men', 5.32896),
 (u'House Royce', 5.328),
 (u'House of Eraz', 5.32727),
 (u'House Hetherspoon', 5.32528),
 (u'House Baratheon', 5.32526),
 (u'Sealord of Braavos', 5.32484),
 (u'Petyr Baelish', 5.32398),
 (u'Volantis', 5.32376),
 (u'House Baelish', 5.32371),
 (u'House Seaworth', 5.32366),
 (u'House Arryn', 5.32365),
 (u'Jon Snow'

# Prediciton of sentiment in season 6 episodes

In [26]:
#Get the character sentiment scores. But only using the first 5 seasons
charnamesopen= open('char_names.txt', 'r')
charlist=charnamesopen.read().splitlines()
charnamesopen.close()
urlnamesopen= open('url_names.txt', 'r')
urllist=urlnamesopen.read().splitlines()
urlnamesopen.close()
season = 0
sentdictpred={}
i=0
# Give the character a sentiment score if they appear 3 or more times.
while season<6:
    if episodesname[i][-5:] == "recap":
        i+=1
        continue
    if episodesname[i][:6] == "Season":
        season += 1
        i+=1
        print "-----season %d-----" % season
        continue
    if season != 0:
        #Use the clean text for the sentiment score put the html text for finding characters.
        text = "Episodes/%s.txt" % episodesname[i]
        text1 = open("Episodes_api/%s.txt" % episodesname[i]).read()
        textsent=sentiment(text)
        for p in range(len(charlist)):
            url=urllist[p]
            regexname=r"\/" + re.escape(url) + r"\b"
            match=re.findall(regexname, text1)
            if len(match)>=3:
                if charlist[p] not in sentdictpred.keys():
                    sentdictpred[charlist[p]]=[]
                sentdictpred[charlist[p]].append(textsent)
    i+=1
print "done"

-----season 1-----
-----season 2-----
-----season 3-----
-----season 4-----
-----season 5-----
-----season 6-----
done


In [53]:
#Create the averaged dictionary
charkeys=sentdictpred.keys()
chardictmeanpred={}
for name in charkeys:
    chardictmeanpred[name]=float("{0:.5f}".format(np.mean(sentdictpred[name])))
    
char_sent_pred_sorted=sorted(chardictmeanpred.items(), key=operator.itemgetter(1),reverse=True)
char_sent_pred_sorted[:10]

[('Jhiqui', 5.38299),
 ('Wylla', 5.38299),
 ('Lady', 5.38299),
 ('Catspaw assassin', 5.38299),
 ('Lothar Frey', 5.37487),
 ('Prendahl na Ghezn', 5.37063),
 ('Mero', 5.37063),
 ('Pentoshi servant', 5.36387),
 ('Waymar Royce', 5.36387),
 ('Wildling girl', 5.36387)]

In [72]:
#Predict sentiment of episodes

season=0
episodeprediction=[]


for i in range(len(episodesname)):
    if episodesname[i][-5:] == "recap":
        continue
    if episodesname[i][:6] == "Season":
        season += 1
        print "-----season %d-----" % season
        continue
    if season == 6:
        episodeprediction.append([])
        text1 = open("Episodes_api/%s.txt" % episodesname[i]).read()
        
        #Append the sentiment score of a character to the given episode. Add it as many times as the characters name appears.
        for p in range(len(charlist)):
            if charlist[p] in sentdictpred.keys():
                url=urllist[p]
                regexname=r"\/" + re.escape(url) + r"\b"
                match=re.findall(regexname, text1)
                for i in range(len(match)):
                    episodeprediction[-1].append(chardictmean[charlist[p]])
            
            
print "done"

-----season 1-----
-----season 2-----
-----season 3-----
-----season 4-----
-----season 5-----
-----season 6-----
done


In [73]:
#Average the sentiments for the episodes and print it.
for i in range(len(episodeprediction)):
    print np.mean(episodeprediction[i])

5.3147491716
5.31665096154
5.31402753012
5.31565245509
5.31575372414


In [33]:
#Get the sentiment score of the episodes in season 6

season = 0
# Give the character a sentiment score if they appear more than x times.
for i in range(len(episodesname)):
    if season==6:
        text = "Episodes/%s.txt" % episodesname[i]
        print "%s: %f" % (episodesname[i], float("{0:.5f}".format(sentiment(text)))) 
    if episodesname[i][-5:] == "recap":
        continue
    if episodesname[i][:6] == "Season":
        season += 1
        print "-----season %d-----" % season
        continue

-----season 1-----
-----season 2-----
-----season 3-----
-----season 4-----
-----season 5-----
-----season 6-----
The Red Woman: 5.324950
Home: 5.298740
Oathbreaker: 5.299020
Book of the Stranger: 5.337710
The Door: 5.309800
